# CS 229 Final Project Notebook 

### Overview

Implementation of LeNet-5 and similar architectures to evaluate complex- and split-complex-valued neural networks. Network is implemented in TensorFlow, and wrappers for complex and split-complex valued implementations are given in layersHDA.py ("layers high-dimensional algebra"). 

### High-dimensional Algebras

To implement layers with complex- and split-complex-valued weights, we can store the weights for the real/imaginary channels separately and use weight-sharing between operations. This is implemented with wrapper files for TensorFlow commands. 

### References
Code implemented here was based on open source code available at: https://github.com/sujaybabruwad/LeNet-in-Tensorflow. The functions to extract CIFAR-10, CIFAR-100, and SVHN were based on code available at: https://github.com/tflearn/tflearn/blob/master/tflearn/datasets/cifar10.py.


In [2]:
# Front matter: load libraries needed for training
import numpy as np
import tensorflow as tf
from utils import *

import os

%reload_ext autoreload
%autoreload 2

In [3]:
# Runfile preliminaries
networkSizes = ['Baseline', 'Wide','Deep']
networkTypes = ['Real', 'Complex','SplitComplex']

# MNIST Training

The cells below load MNIST data and execute run files for the various network architectures. Each iteration saves loss/accuracy curves for the training, validation, and test sets, and the model at the end of training. Models are trained for 20 epochs with a batch size of 200. 

In [11]:
# Initialize MNIST Data

dataset = 'MNIST' #set book-keeping parameter

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", reshape=False)
X_train, y_train           = mnist.train.images, mnist.train.labels
X_validation, y_validation = mnist.validation.images, mnist.validation.labels
X_test, y_test             = mnist.test.images, mnist.test.labels

# Zero-pad MNIST images to be 32x32
X_train      = np.pad(X_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
X_validation = np.pad(X_validation, ((0,0),(2,2),(2,2),(0,0)), 'constant')
X_test       = np.pad(X_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')
    
print("Updated Image Shape: {}".format(X_train[0].shape))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Updated Image Shape: (32, 32, 1)


In [12]:
# Set training parameters
EPOCHS = 40
BATCH_SIZE = 200
NUM_TRAIN = X_train.shape[0] # set number of training examples
NUM_VAL = X_validation.shape[0] 
NUM_TEST = X_test.shape[0] 

In [15]:
# Loop over sizes and types to generate data

for networkSize in networkSizes:
    for networkType in networkTypes:
        runmodel((X_train,y_train,NUM_TRAIN), (X_validation,y_validation,NUM_VAL), (X_test,y_test,NUM_TEST), 
                 dataset, BATCH_SIZE, EPOCHS, num_classes = 10, networkSize = networkSize, networkType = networkType, alpha = 1e-3)

ValueError: Function has keyword-only arguments or annotations, use getfullargspec() API which can support them

# CIFAR-10 Training
Cells below load the CIFAR-10 dataset (https://www.cs.toronto.edu/~kriz/cifar.html), setup the training parameters, and make calls to train the 9 models we are interested in. 

In [4]:
# Initialize CIFAR-10 Data
# Used code from: https://github.com/tflearn/tflearn/blob/master/tflearn/datasets/cifar10.py
import cifar10
dataset = 'CIFAR10' #set book-keeping parameter

train_tup, test_tup = cifar10.load_data()

X_train, y_train = train_tup
X_validation = X_train[49000:,:]
y_validation = y_train[49000:]
X_train = X_train[:49000,:]
y_train = y_train[:49000]

X_test, y_test = test_tup

In [5]:
# Set training parameters
EPOCHS = 100
BATCH_SIZE = 200
NUM_TRAIN = X_train.shape[0] # set number of training examples
NUM_VAL = X_validation.shape[0] 
NUM_TEST = X_test.shape[0] 
print('Number of training examples: {}'.format(NUM_TRAIN))
print('Number of validation examples: {}'.format(NUM_VAL))
print('Number of testing examples: {}'.format(NUM_TEST))

Number of training examples: 49000
Number of validation examples: 1000
Number of testing examples: 10000


In [6]:
# Runfile preliminaries
networkSizes = ['Baseline', 'Wide','Deep']
networkTypes = ['Real', 'Complex','SplitComplex']

In [7]:
# Loop over sizes and types to generate data

for networkSize in networkSizes:
    for networkType in networkTypes:
        runmodel((X_train,y_train,NUM_TRAIN), (X_validation,y_validation,NUM_VAL), (X_test,y_test,NUM_TEST), dataset, 
                 BATCH_SIZE, EPOCHS, num_classes = 10, networkSize = networkSize, networkType = networkType, alpha = 1e-4)

Training...

EPOCH 1 ...
Training Accuracy = 0.225
Validation Accuracy = 0.215

EPOCH 2 ...
Training Accuracy = 0.325
Validation Accuracy = 0.326

EPOCH 3 ...
Training Accuracy = 0.440
Validation Accuracy = 0.384

EPOCH 4 ...
Training Accuracy = 0.410
Validation Accuracy = 0.391

EPOCH 5 ...
Training Accuracy = 0.460
Validation Accuracy = 0.455

EPOCH 6 ...
Training Accuracy = 0.470
Validation Accuracy = 0.485

EPOCH 7 ...
Training Accuracy = 0.480
Validation Accuracy = 0.498

EPOCH 8 ...
Training Accuracy = 0.500
Validation Accuracy = 0.492

EPOCH 9 ...
Training Accuracy = 0.565
Validation Accuracy = 0.525

EPOCH 10 ...
Training Accuracy = 0.560
Validation Accuracy = 0.540

EPOCH 11 ...
Training Accuracy = 0.655
Validation Accuracy = 0.562

EPOCH 12 ...
Training Accuracy = 0.555
Validation Accuracy = 0.548

EPOCH 13 ...
Training Accuracy = 0.650
Validation Accuracy = 0.554

EPOCH 14 ...
Training Accuracy = 0.620
Validation Accuracy = 0.550

EPOCH 15 ...
Training Accuracy = 0.635
Valid

# CIFAR-10 Training (with regularization)
Cells below load the CIFAR-10 dataset (https://www.cs.toronto.edu/~kriz/cifar.html), setup the training parameters, and make calls to train the 9 models we are interested in. Here, we also employ L2 regularization.

In [8]:
# Initialize CIFAR-10 Data
# Used code from: https://github.com/tflearn/tflearn/blob/master/tflearn/datasets/cifar10.py
import cifar10
dataset = 'CIFAR10-L2reg' #set book-keeping parameter

train_tup, test_tup = cifar10.load_data()

X_train, y_train = train_tup
X_validation = X_train[49000:,:]
y_validation = y_train[49000:]
X_train = X_train[:49000,:]
y_train = y_train[:49000]

X_test, y_test = test_tup

In [11]:
# Set training parameters
EPOCHS = 100
BATCH_SIZE = 200
NUM_TRAIN = X_train.shape[0] # set number of training examples
NUM_VAL = X_validation.shape[0] 
NUM_TEST = X_test.shape[0] 

print('Number of training examples: {}'.format(NUM_TRAIN))
print('Number of validation examples: {}'.format(NUM_VAL))
print('Number of testing examples: {}'.format(NUM_TEST))

# Runfile preliminaries
networkSizes = ['Wide','Deep'] # ['Baseline', 'Wide','Deep']
networkTypes = ['Real', 'Complex','SplitComplex']

Number of training examples: 49000
Number of validation examples: 1000
Number of testing examples: 10000


In [15]:
# Loop over sizes and types to generate data

for networkSize in networkSizes:
    for networkType in networkTypes:
        runmodel((X_train,y_train,NUM_TRAIN), (X_validation,y_validation,NUM_VAL), (X_test,y_test,NUM_TEST), 
                 dataset, BATCH_SIZE, EPOCHS, num_classes = 10, networkSize = networkSize, 
                 networkType = networkType, alpha = 5e-4, lam = 5e-3)

Training...

EPOCH 1 ...
Training Accuracy = 0.395
Validation Accuracy = 0.388

EPOCH 2 ...
Training Accuracy = 0.390
Validation Accuracy = 0.436

EPOCH 3 ...
Training Accuracy = 0.445
Validation Accuracy = 0.457

EPOCH 4 ...
Training Accuracy = 0.465
Validation Accuracy = 0.481

EPOCH 5 ...
Training Accuracy = 0.495
Validation Accuracy = 0.472

EPOCH 6 ...
Training Accuracy = 0.465
Validation Accuracy = 0.499

EPOCH 7 ...
Training Accuracy = 0.475
Validation Accuracy = 0.487

EPOCH 8 ...
Training Accuracy = 0.465
Validation Accuracy = 0.494

EPOCH 9 ...
Training Accuracy = 0.485
Validation Accuracy = 0.514

EPOCH 10 ...
Training Accuracy = 0.510
Validation Accuracy = 0.524

EPOCH 11 ...
Training Accuracy = 0.530
Validation Accuracy = 0.524

EPOCH 12 ...
Training Accuracy = 0.525
Validation Accuracy = 0.517

EPOCH 13 ...
Training Accuracy = 0.475
Validation Accuracy = 0.520

EPOCH 14 ...
Training Accuracy = 0.615
Validation Accuracy = 0.531

EPOCH 15 ...
Training Accuracy = 0.555
Valid

# CIFAR-100 Training
Cells below load the CIFAR-100 dataset (https://www.cs.toronto.edu/~kriz/cifar.html), setup the training parameters, and make calls to train the 9 models we are interested in. (This code is nearly identical to the CIFAR-10 cells above.)

In [8]:
# Initialize CIFAR-100 Data
# Used code from: https://github.com/tflearn/tflearn/blob/master/tflearn/datasets/cifar10.py
import cifar100
dataset = 'CIFAR100' #set book-keeping parameter

train_tup, test_tup = cifar100.load_data()

X_train, y_train = train_tup
X_validation = X_train[49000:,:]
y_validation = y_train[49000:]
X_train = X_train[:49000,:]
y_train = y_train[:49000]

X_test, y_test = test_tup

In [34]:
# Set training parameters
EPOCHS = 50
BATCH_SIZE = 200
NUM_TRAIN = X_train.shape[0] # set number of training examples
NUM_VAL = X_validation.shape[0] 
NUM_TEST = X_test.shape[0] 
print('Number of training examples: {}'.format(NUM_TRAIN))
print('Number of validation examples: {}'.format(NUM_VAL))
print('Number of testing examples: {}'.format(NUM_TEST))

Number of training examples: 49000
Number of validation examples: 1000
Number of testing examples: 10000


In [36]:
# Loop over sizes and types to generate data

for networkSize in networkSizes:
    for networkType in networkTypes:
        runmodel((X_train,y_train,NUM_TRAIN), (X_validation,y_validation,NUM_VAL), (X_test,y_test,NUM_TEST), dataset, 
                 BATCH_SIZE, EPOCHS, num_classes = 100, networkSize = networkSize, networkType = networkType, alpha = 5e-4)

Training...

EPOCH 1 ...
Validation Accuracy = 0.066

EPOCH 2 ...
Validation Accuracy = 0.110

EPOCH 3 ...
Validation Accuracy = 0.134

EPOCH 4 ...
Validation Accuracy = 0.155

EPOCH 5 ...
Validation Accuracy = 0.161

EPOCH 6 ...
Validation Accuracy = 0.174

EPOCH 7 ...
Validation Accuracy = 0.172

EPOCH 8 ...
Validation Accuracy = 0.196

EPOCH 9 ...
Validation Accuracy = 0.208

EPOCH 10 ...
Validation Accuracy = 0.206

EPOCH 11 ...
Validation Accuracy = 0.211

EPOCH 12 ...
Validation Accuracy = 0.224

EPOCH 13 ...
Validation Accuracy = 0.224

EPOCH 14 ...
Validation Accuracy = 0.240

EPOCH 15 ...
Validation Accuracy = 0.247

EPOCH 16 ...
Validation Accuracy = 0.231

EPOCH 17 ...
Validation Accuracy = 0.251

EPOCH 18 ...
Validation Accuracy = 0.246

EPOCH 19 ...
Validation Accuracy = 0.248

EPOCH 20 ...
Validation Accuracy = 0.243

EPOCH 21 ...
Validation Accuracy = 0.255

EPOCH 22 ...
Validation Accuracy = 0.249

EPOCH 23 ...
Validation Accuracy = 0.249

EPOCH 24 ...
Validation Accura

# SVHN Training
Cells below load the SVHN dataset (http://ufldl.stanford.edu/housenumbers/), setup the training parameters, and make calls to train the 9 models we are interested in. (This code is nearly identical to the CIFAR-10 cells above.)

In [3]:
# Initialize SVHN Data
# Used code from: https://github.com/tflearn/tflearn/blob/master/tflearn/datasets/cifar10.py
import svhn
dataset = 'SVHN' #set book-keeping parameter

train_tup, test_tup = svhn.load_data()

X_train, y_train = train_tup
X_validation = X_train[72257:]
y_validation = y_train[72257:]
X_train = X_train[:60000]
y_train = y_train[:60000]

X_test, y_test = test_tup


In [4]:
# Set training parameters
EPOCHS = 100
BATCH_SIZE = 128
NUM_TRAIN = X_train.shape[0] # set number of training examples
NUM_VAL = X_validation.shape[0] 
NUM_TEST = X_test.shape[0] 
print('Number of training examples: {}'.format(NUM_TRAIN))
print('Number of validation examples: {}'.format(NUM_VAL))
print('Number of testing examples: {}'.format(NUM_TEST))

Number of training examples: 60000
Number of validation examples: 1000
Number of testing examples: 26032


In [5]:
# Loop over sizes and types to generate data

for networkSize in networkSizes:
    for networkType in networkTypes:
        runmodel((X_train,y_train,NUM_TRAIN), (X_validation,y_validation,NUM_VAL), (X_test,y_test,NUM_TEST), dataset, 
                 BATCH_SIZE, EPOCHS, num_classes = 10, networkSize = networkSize, networkType = networkType, alpha = 1e-1)

Training...

EPOCH 1 ...
Training Accuracy = 0.146
Validation Accuracy = 0.172

EPOCH 2 ...
Training Accuracy = 0.177
Validation Accuracy = 0.172



KeyboardInterrupt: 